In [ ]:
!pip install beautifulsoup4
import nltk
import re 
import bs4
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tensorflow import keras
from gensim.models import Word2Vec
import numpy as np 
from sklearn.model_selection import train_test_split
import pandas as pd 
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import LinearSVC
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
data=pd.read_csv('../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv',names=['review', 'sentiment'],
                   header=0,
                   encoding='utf-8')

In [ ]:
data.head(3)

data_train, data_test, train_data_label, test_data_label = train_test_split(data['review'], data['sentiment'], test_size=0.25, random_state=42)

train_data_label = (train_data_label.replace({'positive': 1, 'negative': 0})).values
test_data_label  = (test_data_label.replace({'positive': 1, 'negative': 0})).values

#train_data_label[1]

In [ ]:
corpus_train = []
corpus_test  = []

for i in range(data_train.shape[0]):
    soup = bs4.BeautifulSoup(data_train.iloc[i], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_train.append(review)
    
for j in range(data_test.shape[0]):
    soup = bs4.BeautifulSoup(data_test.iloc[j], "html.parser")
    review = soup.get_text()
    review = re.sub('\[[^]]*\]', ' ', review)
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    review = [word for word in review if not word in set(stopwords.words('english'))]
    lem = WordNetLemmatizer()
    review = [lem.lemmatize(word) for word in review]
    review = ' '.join(review)
    corpus_test.append(review)










In [ ]:
tfidf = TfidfVectorizer(ngram_range=(1, 3))

tfidf_train = tfidf.fit_transform(corpus_train)
tfidf_test = tfidf.transform(corpus_test)

In [ ]:


linear_svc = LinearSVC(C=0.5, random_state=42)
linear_svc.fit(tfidf_train, train_data_label)

predict = linear_svc.predict(tfidf_test)

In [ ]:


print("Classification Report: \n", classification_report(test_data_label, predict,target_names=['Negative','Positive']))
print("Confusion Matrix: \n", confusion_matrix(test_data_label, predict))
print("Accuracy: \n", accuracy_score(test_data_label, predict))